1.感知机（perceptron）是根据输入实例的特征向量x对其进行二分类的线性分类模型
$$f(x) = sign(w * x + b)$$
  感知机模型对应于输入空间（特征空间）中的分离超平面$w * x + b = 0$

2.感知机学习的策略是极小化损失函数：
  $$\min_{w,b}L(a,b) = - \sum_{x_i\in M}y_i(w * x_i + b)$$
  损失函数对应于误分类点到分离超平面的总距离

3.感知机学习算法是基于随机梯度下降的对损失函数的最优化算法，有原始形式和对偶形式。算法简单且易于实现。原始形式中，首先任意选取一个超平面，然后用梯度下降法不断极小化目标函数。在这个过程中一次随机选取一个误分类点使其梯度下降

4.当训练数据集线性可分时，感知机学习算法是收敛的。（Novikoff）
  当训练数据集线性可分时，感知机学习算法存在无穷多个解，其解由于不同的初值或不同的迭代顺序而可能有所不同

举例说明
给定集合点以及label

In [1]:
import numpy as np
x = [[3,3],[4,3],[1,1]]
y = [[1],[1],[-1]]
w = [0,0]
b = 0
x = np.array(x)
y = np.array(y)
w = np.array(w)

损失函数 $$L(a,b) = - \sum_{x_i\in M}y_i(w * x_i + b)$$

考虑一下损失函数的梯度
$$\nabla_wL(w,b) = - \sum_{x_i\in M}y_ix_i$$
$$\nabla_bL(w,b) = - \sum_{x_i\in M}y_i$$
如果每次更新只选取一个点$(x_i,y_i)$
那么更新公式就是
$$w = w + \eta y_ix_i$$
$$b = b + \eta y_i$$

In [2]:
def train(x,y,w,b,eta):
    while True:
        i = find_false(x,y,w,b)
        if i is True:
            break
        w = w + eta*y[i]*x[i]
        b = b + eta*y[i]
    return w,b

In [3]:
def find_false(x,y,w,b):
    for i in range(len(x)):
        if y[i]*(np.dot(x[i],w) + b) <= 0:
            return i
    return True
                   

In [43]:
eta = 1
w,b = train(x,y,w,b,eta)
print(w)
print(b)

[1. 1.]
[-3]


考虑其对偶形式
对偶形式的基本想法是，将$w$和$b$表示为实例$x_i$和$y_i$的线性组合的形式，通过求解其系数而求得$w$和$b$。$w$和$b$原更新公式：
$$w = w + \eta y_ix_i$$
$$b = b + \eta y_i$$
逐步修改$w$和$b$,设修改$n$次，则$w$和$b$关于$(x_i,y_i)$的增量分别是$\alpha_iy_ix_i$和$\alpha_iy_i$，这里$\alpha_i = n_i\eta$ ,这样$w$和$b$也可以表示为
$$w = \sum_{i=1}^N\alpha_iy_ix_i$$
$$b = \sum_{i=1}^N\alpha_iy_i$$
这里$\alpha_i >=0,i=1,2,...N,当$\ate = 1$时，表示第$i$个实例点由于误分而进行更新的次数。实例点更新次数越多，意味着它距离分离超平面越近，也就越难正确分类。换句话说，这样的实例对学习结果影响最大。


$$f(x) = sign(w \cdot x + b) -->  f(x) = sign(\sum_{j=1}^N\alpha_iy_ix_i \cdot x + b)$$

定义Gram矩阵 $$G = [x_i \cdot x_j ]_{N\times N}$$

In [28]:
len(G)

3

In [37]:
def find_false_dual(x,y,alpha,G,b):
    for i in range(len(x)):
        if y[i]*(sum(alpha[k] * y[k] * G[k][i] for k in range(len(G))) + b) <= 0:
            return i
    return True

In [38]:
def train_dual(x,y,b,eta):
    G = np.array(np.mat(x) * np.transpose(np.mat(x)))
    alpha = np.zeros(len(x))
    while True:
        i = find_false_dual(x,y,alpha,G,b)
        if i is True:
            break
        alpha[i] = alpha[i] + eta
        b = b + eta*y[i]
    return alpha,b

In [47]:
b = 0
alpha,b = train_dual(x,y,b,eta)
w = sum(alpha[i] * x[i] * y[i] for i in range(len(x)))
print(alpha)
print(w)
print(b)

[2. 0. 5.]
[1. 1.]
[-3]
